### After cleaning the initial dataset, we are going to look at enriching it with a key variable which is going to be used in some models as the predictor variable, which is company valuation

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
pd.set_option("display.max_columns", None)

In [2]:
startups = pd.read_csv("../../dataset/Cleaned_startups.csv", index_col=0)
url_dr = "https://app.dealroom.co/companies/"

In [3]:
startups.head()

,permalink,name,market,funding_total_usd,status,country_code,city,funding_rounds,founded_at,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,delta_founded_vs_firstfunding
0,/organization/waywire,#waywire,News,1750000,acquired,USA,New York,1.0,2012-06-01,2012.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0
3,/organization/in-touch-network,(In)Touch Network,Electronics,1500000,operating,GBR,London,1.0,2011-04-01,2011.0,2011-04-01,2011-04-01,1500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,Tourism,60000,operating,USA,Fort Worth,2.0,2014-01-01,2014.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,228.0
8,/organization/004-technologies,004 Technologies,Software,0,operating,USA,Champaign,1.0,2010-01-01,2010.0,2014-07-24,2014-07-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1665.0
10,/organization/1-2-3-listo,"1,2,3 Listo",E-Commerce,40000,operating,CHL,Las Condes,1.0,2012-01-01,2012.0,2013-02-18,2013-02-18,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,414.0


### We will get the valuation from Deal Room webpage, I will set a for loop, but will do an individual try first

In [4]:
from selenium import webdriver
from time import sleep

In [5]:
driver = webdriver.Chrome("../../tools/Selenium/chromedriver.exe")

In [6]:
driver.get("https://app.dealroom.co/companies")
sleep(2)

In [7]:
from selenium.webdriver.common.keys import Keys

In [8]:
driver.find_elements_by_css_selector("header div div div div div input")[0].send_keys("Field Nation")

In [9]:
driver.find_elements_by_css_selector("header div div div div div input")[0].send_keys(u'\ue007')

In [10]:
driver.find_elements_by_css_selector("div ul li a div h3")[0].click()

In [11]:
driver.find_elements_by_css_selector("span.valuation__value")[0].text

'€109—164m'

In [12]:
driver.close()

### Now we will do the loop to get this value for every company

In [ ]:
startups_valuation = []
for i, row in startups.iterrows():
    try:
        name = row["name"]
        driver = webdriver.Chrome("../../tools/Selenium/chromedriver.exe")
        driver.get("https://app.dealroom.co/companies")
        sleep(1)
        driver.find_elements_by_css_selector("header div div div div div input")[0].send_keys(name)
        sleep(1)
        driver.find_elements_by_css_selector("header div div div div div input")[0].send_keys(u'\ue007')
        sleep(2)
        driver.find_elements_by_css_selector("div ul li a div h3")[0].click()
        startups_valuation.append(driver.find_elements_by_css_selector("span.valuation__value")[0].text)
        driver.close()
        sleep(3)
    except:
        startups_valuation.append(0)

### I have had trouble with Selenium as the web has detected it's a robot scraping so I will be using Beatiful Soup instead.  However, the code for the loop would look like the above

### First, let's define some parsers we will use for the web browser links when including the company name at the end of it

In [13]:
def parser1(name):
    return name.lower().replace(" ", "")

In [15]:
parser1("My Startup")

'mystartup'

In [17]:
def parser2(name):
    return name.lower().replace(" ", "-")

In [18]:
parser2("My Startup")

'my-startup'

In [19]:
def parser3(name):
    return name.lower().replace(" ", "_")

In [20]:
parser3("My Startup")

'my_startup'

In [21]:
parsers = [parser1, parser2, parser3]

In [22]:
def obtain_valuation_parser(name, parser):
    url = url_dr + parser(name)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    company_funding = soup.find(name="span", class_="valuation__value").text
    
    return company_funding

In [23]:
obtain_valuation_parser("Acclaimd", parser1)

'€255—382k'

### Let's make it so that the function can run through the 3 parsers and look for the valuation

In [24]:
def obtain_valuation(name):
    # tries all parsers in the list until one works. Then prints info and returns valuation
    # else returns None and prints Nope
    for p in parsers:
        try:
            val = obtain_valuation_parser(name, p)
            print(f"{name:30} used {p.__name__:10} valuation: {val}")
            return val
        except:
            continue
    else:
        print(f"{name:30} {'Nope':10}")

In [25]:
obtain_valuation("My Startup")

My Startup                     Nope      


In [26]:
obtain_valuation("Acclaimd")

Acclaimd                       used parser1    valuation: €255—382k


'€255—382k'

In [27]:
obtain_valuation("Bee Cave Games")

Bee Cave Games                 used parser3    valuation: €20—30m


'€20—30m'

### Let's apply the function to the whole dataset now

In [29]:
import numpy as np

In [ ]:
startups["valuation"] = np.nan
for i, s in startups.iterrows():
    v = obtain_valuation(s["name"])
    startups.loc[i, "valuation"] = v
    if i % 10 == 0:
        startups.to_csv("startups_with_valuation_temp.csv")
        print(f"saved at index {i}")

#waywire                       Nope      
saved at index 0
(In)Touch Network              Nope      
-R- Ranch and Mine             Nope      
004 Technologies               Nope      
1,2,3 Listo                    Nope      
saved at index 10
1-800-DENTIST                  Nope      
1-800-DOCTORS                  Nope      
1.618 Technology               Nope      
10 Minutes With                used parser3    valuation: €14—22m
10-20 Media                    Nope      
1000 Corks                     used parser3    valuation: €145—218k
1000museums.com                Nope      
1001 Menus                     Nope      
1006.tv                        Nope      
100health                      Nope      
100Plus                        used parser1    valuation: €91—136m
1010data                       used parser1    valuation: €140—210m
10�North                       Nope      
10BestThings                   used parser1    valuation: €182—273k
saved at index 30
10seconds Software    

3Pillar Global                 Nope      
3Play Media                    Nope      
3ROAM                          used parser1    valuation: €4—7m
3scale                         used parser1    valuation: €3—4m
3Scan                          used parser1    valuation: €24—36m
saved at index 250
3seventy                       Nope      
3Sourcing                      Nope      
3TEN8                          Nope      
3TIER                          used parser1    valuation: €13m
3Touch                         used parser1    valuation: €982k—1m
3V Transaction Services        used parser3    valuation: €15m
3VR                            used parser1    valuation: €29—44m
3X Systems                     used parser3    valuation: €6—9m
saved at index 260
4 the stars                    used parser3    valuation: €4—5k
4-Tell                         Nope      
405Labs                        Nope      
40billion.com                  Nope      
410 Labs                       used parser3  

AAIPharma Services             used parser3    valuation: €190m
AAMPP                          Nope      
Aarden Pharmaceuticals         used parser3    valuation: €2—3m
Aaron Andrews Apparel          Nope      
Aasonn                         used parser1    valuation: €127—182m
Aastrom Biosciences            Nope      
aaTag                          used parser1    valuation: €722k—1m
saved at index 510
Aava Mobile                    used parser3    valuation: €15—23m
AB Tasty                       used parser3    valuation: €145—218m
ab&jb properties and services  Nope      
ABA English                    used parser3    valuation: €44—65m
Abacast                        used parser1    valuation: €2—3m
saved at index 520
Abacus e-Media                 Nope      
Abacus Labs                    Nope      
Abakan                         Nope      
Abakus                         used parser1    valuation: €3—5m
ABARIS Investment Management   Nope      
Abaxia                         used

Accord Biomaterials            used parser3    valuation: €4—7m
Accounting SaaS Japan          Nope      
AccountNow                     used parser1    valuation: €47—70m
Accruent                       used parser1    valuation: €1.8b
Accruit                        used parser1    valuation: €12—18m
Accu-Break Pharmaceuticals     Nope      
saved at index 710
Accudial Pharmaceutical        Nope      
AccuDraft                      used parser1    valuation: €2—4m
Accuhealth Partners            Nope      
AccuMED Technologies           Nope      
Accumetrics                    used parser1    valuation: €38—55m
Accumulate                     used parser1    valuation: €36—55m
Accumuli Security              used parser3    valuation: €24—36m
AccuNostics                    used parser1    valuation: €19—29m
Accupal                        Nope      
Accuradio                      used parser1    valuation: €9—14m
Accurence                      used parser1    valuation: €9—13m
AccuRev    

ActualMeds                     used parser1    valuation: €4—7m
ActuatedMedical                used parser1    valuation: €545—818k
Acturis                        Nope      
Actus Digital                  used parser3    valuation: €7—11m
Actus Interactive Software     Nope      
ACTV8me                        Nope      
ActX                           used parser1    valuation: €3—4m
Acucar Guarani                 used parser3    valuation: €165—248m
Acucela                        used parser1    valuation: €2—2m
saved at index 910
AcuFocus                       used parser1    valuation: €76—115m
Acuity Medical International   used parser3    valuation: €5—8m
AcuityAds                      used parser1    valuation: €545m
Acumatica                      used parser1    valuation: €91—136m
Acumen Holdings                used parser3    valuation: €528—755m
Acumen Pharmaceuticals         used parser3    valuation: €73—109m
Acumentrics                    Nope      
saved at index 920
Acun

...

Dyyno                          used parser1    valuation: €11—16m
DZZOM                          Nope      
e Health Access                used parser3    valuation: €2—3m
E Ink                          used parser3    valuation: €58—87m
saved at index 12550
E Ink Holdings                 used parser3    valuation: €2.4b
E la Carte                     Nope      
E&E Enterprises Global         Nope      
e(ye)BRAIN                     Nope      
E-Band Communications          Nope      
e-Booking.com                  Nope      
E-Box - Blogo.it               Nope      
E-Buy                          Nope      
E-Car Club                     Nope      
saved at index 12560
e-Chromic Technologies         Nope      
e-contratos                    Nope      
E-Cube Energy                  Nope      
E-Diversify Yourself           Nope      
E-Drive Autos                  Nope      
E-Duction                      Nope      
e-Go aeroplanes                Nope      
E-Health Records Internati

Echo360                        used parser1    valuation: €11—16m
echoBase                       used parser1    valuation: €3—5m
Echobit                        Nope      
saved at index 12770
Echobot Media Technologies GmbH used parser3    valuation: €3—5m
echoecho                       used parser1    valuation: €3—4m
EchoFirst                      used parser1    valuation: €50—75m
Echogen Power Systems          used parser3    valuation: €364—545k
Echoing Green                  used parser3    valuation: €364—545k
Echolocation                   used parser1    valuation: €727k—1m
Echopass Corporation           used parser3    valuation: €22—33m
EchoPixel                      used parser1    valuation: €31—46m
EchoSign                       used parser1    valuation: €22—33m
Echovox                        used parser1    valuation: €27—41m
ECI Telecom                    Nope      
Ecinity                        used parser1    valuation: €41—61m
eCircle                        used p

EDUonGo                        used parser1    valuation: €2—3m
eduPad                         used parser1    valuation: €800k—1m
Edupath                        Nope      
edupristine                    used parser1    valuation: €36—55m
EduRise                        used parser1    valuation: €145—218k
saved at index 13000
EDUS                           used parser1    valuation: €2—3m
Edusoft                        used parser1    valuation: €4—5m
EduSourced                     used parser1    valuation: €982k—1m
Edustation.me                  Nope      
Edventory                      used parser1    valuation: €55—82k
Edventures                     used parser1    valuation: €782k—1m
Edvisor.io                     Nope      
eelusion                       used parser1    valuation: €6—9m
eEvent                         used parser1    valuation: €909k—1m
eEye                           Nope      
eFans                          used parser1    valuation: €182—273k
Eferio             

Eliason Media                  used parser3    valuation: €364—545k
eLibs.com                      Nope      
elicit                         used parser1    valuation: €2—3m
ELIKE                          Nope      
Elimi                          Nope      
Eline Fleury                   Nope      
Elite Education Media Group    Nope      
Elite Form                     used parser3    valuation: €4—5m
saved at index 13240
Elite Meetings International   used parser3    valuation: €735k—1m
Elite Motorcycle Parts         Nope      
Elite Pharmaceuticals          used parser3    valuation: €764k—1m
Elivar                         used parser1    valuation: €3—5m
Elixent                        used parser1    valuation: €55—82m
Elixir Bio-Tech                Nope      
Elixir Medical                 used parser3    valuation: €19—29m
Elixir Pharmaceuticals         used parser3    valuation: €44—65m
Eliza Corporation              Nope      
Ella Health                    Nope      
Ellacoya 

Endeavour Software Technologies Nope      
Ender Labs                     used parser3    valuation: €5—8m
Enders Fund                    Nope      
Endgame                        used parser1    valuation: €213m
EndGenitor Technologies        used parser3    valuation: €9—14m
Endo Tools Therapeutics        used parser3    valuation: €35—52m
EndoBiologics International    Nope      
EndoChoice                     used parser1    valuation: €191m
Endocrine Technology           Nope      
saved at index 13500
Endocyte                       used parser1    valuation: €1.9b
EndoEvolution                  used parser1    valuation: €18—27m
EndoGastric Solutions          used parser3    valuation: €55—82m
Endoluminal Sciences           used parser3    valuation: €9—14m
EndoMetabolic Solutions        Nope      
Endomondo                      used parser1    valuation: €77m
Endonovo Therapeutics          Nope      
Endorphin                      used parser1    valuation: €909k—1m
Endorse     

In [ ]:
startups["valuation_dealroom"] = startups.name.apply(obtain_valuation)